In [18]:
import healpy as hp
import numpy as np
import os
import time
import multiprocessing as mproc
from lsst.daf.persistence import Butler
repo_dir = '/datasets/hsc/repo/rerun/DM-13666/WIDE'
butler = Butler(repo_dir)
#/datasets/hsc/repo/rerun/DM-13666/DEEP/deepCoadd-results/HSC-R

In [45]:
def get_pixel_counts(butler, catalog_name, dataId_list, nside,
                     i_process, out_dict):
    """
    Takes a butler and a list of dataIds and figurs out how many
    stars appear in each healpix
    
    Parameters
    ----------
    butler -- an instance of lsst.daf.persistence.Butler
    
    catalog_name -- the dataset to query (i.e. 'deepCoadd_ref')
    
    dataId_list -- a list of dataIds to process
    
    nside -- the nside of the healpixel grid
    
    i_process -- a unique identifier for this process as called
    from multiprocessing
    
    out_dict -- a multiprocessing.Manager().dict() where we will
    store the output
    
    Returns
    -------
    Nothing
    
    A numpy array corresponding to the number of stars in each
    healpixel is stored in out_dict[i_process]
    """
    
    npix = hp.nside2npix(nside)
    pixel_counts = np.zeros(npix, dtype=int)
    ct = 0
    t_start = time.time()
    for dataId in dataId_list:
        ref_cat = butler.get(catalog_name, dataId=dataId)
        is_primary = ref_cat['detect_isPrimary']
        is_star = ref_cat['base_ClassificationExtendedness_value'] < 0.99
        valid = is_primary & is_star
        data = ref_cat[valid]
        pixel_number = hp.ang2pix(nside,
                                  np.degrees(data['coord_ra']),
                                  np.degrees(data['coord_dec']),
                                  lonlat=True)

        l_pixel_dexes, l_pixel_counts = np.unique(pixel_number,
                                                  return_counts=True)
        pixel_counts[l_pixel_dexes]+=l_pixel_counts
        ct += 1
        if ct %10 ==0:
            duration = time.time()-t_start
            predict = len(dataId_list)*duration/ct
            print('ran %d of %d dataIds; took %.2e sec; total should take %.2e hrs' %
                  (ct, len(dataId_list), duration, predict/3600.0))
    out_dict[i_process] = pixel_counts

In [2]:
nside = 512
npix = hp.nside2npix(nside)
print('resolution %e sq degrees' % (4.0*np.pi*(180.0/np.pi)**2/npix))

resolution 1.311396e-02 sq degrees


In [3]:
pixel_counts = np.zeros(npix, dtype=float)

In [4]:
sky_map = butler.get('deepCoadd_skyMap')

In [5]:
catalog_name = 'deepCoadd_forced_src'
ref_name = 'deepCoadd_ref'
filter_name = 'HSC-R'

In [6]:
tract_root = os.path.join(repo_dir, 'deepCoadd-results', filter_name)
tract_list = os.listdir(tract_root)
tract_list = np.array(tract_list).astype(int)

In [35]:
valid_data_id = []
for tract_id in tract_list:
    tract = sky_map[tract_id]
    patches = tract.getNumPatches()
    for ip1 in range(patches[0]):
        for ip2 in range(patches[1]):
            dataId = {'filter':filter_name,
                      'tract':tract_id,
                      'patch':'%d,%d' % (ip1, ip2)}
            if not butler.datasetExists(catalog_name, dataId=dataId):
                continue
            else:
                valid_data_id.append(dataId)

In [36]:
import copy
print(len(valid_data_id))
valid_data_id_true = copy.deepcopy(valid_data_id)

6965


In [47]:
nside = 512
n_proc = 4
valid_data_id = copy.deepcopy(valid_data_id_true)
manager = mproc.Manager()
pixel_dict = manager.dict()

# just take a random subset
valid_data_id = np.array(valid_data_id)
rng = np.random.RandomState(9912)
rng.shuffle(valid_data_id)
valid_data_id = valid_data_id[:400]

valid_id_arr = []
butler_arr = []

n_frac = len(valid_data_id)//n_proc

print('n_frac %d len %d' % (n_frac,len(valid_data_id)))

for ii in range(n_proc):
    i_start = ii*n_frac
    i_end = (ii+1)*n_frac
    if ii == n_proc-1:
        i_end=len(valid_data_id)
    local_id_list = valid_data_id[i_start:i_end]
    print(i_start,i_end,len(local_id_list))
    valid_id_arr.append(local_id_list)
    butler_arr.append(Butler(repo_dir))

    #butler, catalog_name, dataId_list, nside,
    #                 i_process, out_dict
    
job_list = []
for ii in range(n_proc):
    p = mproc.Process(target=get_pixel_counts,
                      args=(butler_arr[ii],
                            'deepCoadd_ref',
                            valid_id_arr[ii],
                            nside, ii, pixel_dict))

    p.start()
    job_list.append(p)

for p in job_list:
    p.join()

n_frac 100 len 400
0 100 100
100 200 100
200 300 100
300 400 100
ran 10 of 100 dataIds; took 3.06e+01 sec; total should take 8.51e-02 hrs
ran 10 of 100 dataIds; took 3.56e+01 sec; total should take 9.89e-02 hrs
ran 10 of 100 dataIds; took 3.58e+01 sec; total should take 9.93e-02 hrs
ran 10 of 100 dataIds; took 4.07e+01 sec; total should take 1.13e-01 hrs
ran 20 of 100 dataIds; took 7.50e+01 sec; total should take 1.04e-01 hrs
ran 20 of 100 dataIds; took 7.70e+01 sec; total should take 1.07e-01 hrs
ran 20 of 100 dataIds; took 8.31e+01 sec; total should take 1.15e-01 hrs
ran 20 of 100 dataIds; took 8.33e+01 sec; total should take 1.16e-01 hrs
ran 30 of 100 dataIds; took 1.24e+02 sec; total should take 1.15e-01 hrs
ran 30 of 100 dataIds; took 1.27e+02 sec; total should take 1.17e-01 hrs
ran 30 of 100 dataIds; took 1.31e+02 sec; total should take 1.21e-01 hrs
ran 30 of 100 dataIds; took 1.34e+02 sec; total should take 1.24e-01 hrs
ran 40 of 100 dataIds; took 1.62e+02 sec; total should take

In [ ]:
ref_cat = butler.get(ref_name, dataId=valid_data_id[0])
src_cat = butler.get(catalog_name, dataId=valid_data_id[0])

In [ ]:
len(ref_cat)

In [ ]:
len(src_cat)

In [ ]:
np.testing.assert_array_equal(ref_cat['coord_ra'], src_cat['coord_ra'])

In [ ]:
np.testing.assert_array_equal(ref_cat['coord_dec'], src_cat['coord_dec'])

In [ ]:
ref_cat['detect_isPrimary']

In [ ]:
ref_cat.schema.find('base_PsfFlux_flux')

In [ ]:
#src_cat.schema.getNames()

In [ ]:
src_val = src_cat['base_ClassificationExtendedness_value']
ref_val = ref_cat['base_ClassificationExtendedness_value']
flux_flag = ref_cat['modelfit_CModel_flag']
src_psf_flux = src_cat['base_PsfFlux_flux']
src_model_flux = src_cat['modelfit_CModel_flux']
ref_psf_flux = ref_cat['base_PsfFlux_flux']
ref_model_flux =ref_cat['modelfit_CModel_flux']

src_ratio = src_psf_flux/src_model_flux
ref_ratio = ref_psf_flux/ref_model_flux
for ii in range(len(src_cat)):
    if src_val[ii] != ref_val[ii] and not np.isnan(src_val[ii]) and \
    not np.isnan(ref_val[ii]):
        print(src_val[ii], ref_val[ii],
              src_ratio[ii], ref_ratio[ii])

In [ ]:
help(hp.ang2pix)

In [ ]:
import time
ct = 0
pixel_counts = np.zeros(npix, dtype=float)
t_start = time.time()
for dataId in valid_data_id:
    ref_cat = butler.get(ref_name, dataId=dataId)
    is_primary = ref_cat['detect_isPrimary']
    is_star = ref_cat['base_ClassificationExtendedness_value'] < 0.99
    valid = is_primary & is_star
    data = ref_cat[valid]
    pixel_number = hp.ang2pix(nside,
                              np.degrees(data['coord_ra']),
                              np.degrees(data['coord_dec']),
                              lonlat=True)

    l_pixel_dexes, l_pixel_counts = np.unique(pixel_number,
                                              return_counts=True)
    pixel_counts[l_pixel_dexes]+=l_pixel_counts
    ct += 1
    if ct %10 ==0:
        duration = time.time()-t_start
        predict = len(valid_data_id)*duration/ct
        print('ran %d of %d dataIds; took %e sec; total should take %e sec' %
              (ct, len(valid_data_id), duration, predict))

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

In [197]:
valid_pixels = np.where(pixel_counts>0.0)

In [198]:
ra_arr, dec_arr = hp.pix2ang(nside, range(len(pixel_counts)),lonlat=True)

In [ ]:
valid_ra = ra_arr[valid_pixels]
valid_dec = dec_arr[valid_pixels]
valid_counts = pixel_counts[valid_pixels]

ra_min = valid_ra.min()
ra_max = valid_ra.max()
dec_min = valid_dec.min()
dec_max = valid_dec.max()

In [ ]:
print(ra_min,ra_max)
print(dec_min,dec_max)

In [ ]:
plt.figsize=(30,30)

plt.scatter(valid_ra, valid_dec, c=valid_counts)
plt.xlabel('ra')
plt.ylabel('dec')
plt.colorbar()

In [ ]:
cart_view = hp.mollview(pixel_counts, cbar=False,
                        #lonra=[ra_min,ra_max],
                        #latra=[dec_min,dec_max],
                        flip='astro',
                        return_projected_map=True)


plt.imshow(cart_view, extent=[ra_min, ra_max, dec_min, dec_max],
           origin='lower', interpolation='none')
plt.colorbar()
ax = plt.gca()
plt.xlabel('ra')
ax.set_ylabel('dec')
xticks = np.arange(ra_min, ra_max, 0.1*(ra_max-ra_min))
print(cart_view.shape)
print(cart_view.shape[0]*cart_view.shape[1], npix)
print(cart_view)

In [ ]:
hp.mollview(pixel_counts, cbar=False,
                        flip='astro',
                        return_projected_map=True)
hp.graticule()
ax = plt.gca()
im = ax.get_images()[0]
plt.colorbar(im)

In [ ]:
len(valid[0])

In [ ]:
len(pixel_counts)

In [ ]:
np.median(pixel_counts[valid])

In [ ]:
#help(hp.gnomview)

In [ ]:
help(hp.projaxes)

In [ ]:
hp.nside2npix(16)

In [ ]:
help(hp.graticule)